# Topic Modelling - Notes and Playground
(July 06, 2019)

Instead of creating a formal Lit. Review, I would like to learn as much as needed to apply high level concepts for topic modelling. This will start by digging into blogs which utilize standard Python libraries and then push into more in-depth articles, papers, etc. regarding better implementation.

## Table of Contents

1. ["Applying Machine Learning to Classify an Unsupervised Text Documents"](#1)
    * Data -> TD-IDF -> K-means
    
2. ["pyLDAvis - Library for Interactive Topic Model Visualization"](#2)
    * Port of an R package
    
3. ["LDA in Python - How to Grid Search Best Topic Models"](#3)

4. ["Unsupervised LDA to Semi-Supervised GuidedLDA"](#4)

## Import Libraries and Data

In [111]:
# Imports

# Standard Libraries
from itertools import combinations

# Numerical Libraries
import numpy as np
from scipy.stats import skew, kurtosis
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Visual Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

# Local Package Libraries
import sys
sys.path.append("../..")

from src.data.make_dataset import *
from src.features.general_helper_functions import *
from src.features.text_cleaning import *

In [2]:
# Settings

# Stop the warnings for chain in pandas...
pd.options.mode.chained_assignment = None

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

%matplotlib inline

In [3]:
# raw data import
_, watchlist_raw, stock_prices_raw = get_raw_data()
article_raw = clean_and_open_business_wire_data_01(None)

In [4]:
# Base data cleaning and formatting
articles = article_raw.copy()
articles.reset_index(inplace=True)
articles.time = pd.to_datetime(articles.time)

watchlist = watchlist_raw.copy()

<a id="1"></a>
## Applying Machine Learning to Classify an Unsupervised Text Documents

* Author builds a model to classify a data set into 2 classifications.
* Steps:
    1. Vectorize the text data by using sklearn's TfidfVectorizer
    2. Implements sklearn's KMeans cluster algorithm with true_k = 2
* The input document is a list of strings, by the appearance the strings are already relatively clean.

In [5]:
# Import
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [6]:
titles_cleaned = clean_text(articles, "title").title.values

In [7]:
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(titles_cleaned)

print(X.shape)

(8433, 7676)


In [8]:
true_num_topics=2 # Obviously not true, but I will go with it for now

naive_kmeans_topic_model = KMeans(
    n_clusters=true_num_topics, 
    init="k-means++", 
    max_iter=100, 
    n_init=1)

naive_kmeans_topic_model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=2, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

Can now take a look at the top words in each centroid:

In [9]:
order_centroids = naive_kmeans_topic_model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(true_num_topics):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(" %s" % terms[ind])
    print("\n")

Cluster 0:
 announce
 result
 quarter
 financial
 conference
 investor
 pharmaceutical
 report
 present
 healthcare


Cluster 1:
 market
 researchandmarkets
 com
 global
 pipeline
 research
 review
 forecast
 analysis
 drug




<a id="2"></a>
## pyLDAvis - Library for Interactive Topic Model Visualization
https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb#topic=1&lambda=0.5&term=

pyLDAvis is a library that ports an R script for topic visualization.

It is possible to oberlay it with gensim and sklearn.

**pyLDAvis**
https://pyldavis.readthedocs.io/en/latest/

**Gensim's LDA**
https://radimrehurek.com/gensim/models/ldamodel.html

In [95]:
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models.tfidfmodel import TfidfModel
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [112]:
tokenized_titles_cleaned = [remove_short_words(text).split(" ") for text in titles_cleaned]
dictionary = corpora.Dictionary(tokenized_titles_cleaned)
dictionary.filter_extremes(no_below=5, no_above=0.4)

In [113]:
corpus = [dictionary.doc2bow(text) for text in tokenized_titles_cleaned]

In [114]:
tfidf_corpus = TfidfModel(corpus)[corpus]

In [115]:
lda = LdaModel(tfidf_corpus, num_topics=20)

In [116]:
for topic_id in range(4):
    print("Topic:", topic_id)
    print([(dictionary[item[0]], np.round(item[1],3)) for item in lda.get_topic_terms(topic_id)], "\n")

Topic: 0
[('needle', 0.024), ('bone', 0.019), ('free', 0.019), ('system', 0.016), ('advance', 0.014), ('market', 0.014), ('diagnostics', 0.014), ('molecular', 0.013), ('earn', 0.013), ('research', 0.012)] 

Topic: 1
[('quarter', 0.058), ('result', 0.056), ('fourth', 0.047), ('year', 0.047), ('financial', 0.038), ('full', 0.037), ('report', 0.033), ('first', 0.032), ('synthetic', 0.017), ('johnson', 0.013)] 

Topic: 2
[('financial', 0.049), ('quarter', 0.047), ('result', 0.045), ('chief', 0.041), ('officer', 0.04), ('appoint', 0.029), ('report', 0.029), ('operate', 0.029), ('third', 0.027), ('president', 0.027)] 

Topic: 3
[('trend', 0.037), ('forecast', 0.032), ('market', 0.03), ('global', 0.029), ('researchandmarkets', 0.027), ('analysis', 0.025), ('price', 0.022), ('growth', 0.019), ('size', 0.019), ('share', 0.016)] 



In [117]:
pyLDAvis.gensim.prepare(lda, tfidf_corpus, dictionary)

c:\users\michael\documents\projects and portfolio work\biopharmaceuticalfactormodel\venv\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
17     0.123626  0.013776       1        1  7.251188
3      0.146898  0.074478       2        1  7.010685
2     -0.114161 -0.238603       3        1  6.257292
18    -0.202612  0.183061       4        1  5.944338
13     0.155543  0.052596       5        1  5.743040
0      0.071013  0.042178       6        1  5.000749
15     0.019159 -0.073154       7        1  4.941823
16     0.009172  0.015585       8        1  4.918890
6      0.135355  0.056951       9        1  4.885781
1     -0.049762 -0.195324      10        1  4.813283
12    -0.150509  0.115537      11        1  4.797363
14    -0.034978 -0.092459      12        1  4.711220
10    -0.079093 -0.170809      13        1  4.701841
7     -0.239899  0.099560      14        1  4.574577
4      0.147495  0.038143      15        1  4.514194
9      0.102014 -0.033007      16        1  4.339076
8     -0.062337  0.137272      17        1  4.191544
5      0.063833  0.004695      18        1  4.027171
11    -0.017670 -0.057774      19        1  3.697541
19    -0.023089  0.027298      20        1  3.678396, topic_info=     Category        Freq                Term       Total  loglift  logprob
46    Default  156.000000             quarter  156.000000  30.0000  30.0000
16    Default  175.000000              result  175.000000  29.0000  29.0000
44    Default  136.000000           financial  136.000000  28.0000  28.0000
156   Default  164.000000            investor  164.000000  27.0000  27.0000
43    Default  209.000000              review  209.000000  26.0000  26.0000
697   Default   57.000000               ipsen   57.000000  25.0000  25.0000
41    Default  244.000000            pipeline  244.000000  24.0000  24.0000
2     Default  133.000000          conference  133.000000  23.0000  23.0000
172   Default   97.000000                firm   97.000000  22.0000  22.0000
379   Default   74.000000            business   74.000000  21.0000  21.0000
7     Default  115.000000             present  115.000000  20.0000  20.0000
296   Default   76.000000                life   76.000000  19.0000  19.0000
470   Default   47.000000        epidemiology   47.000000  18.0000  18.0000
89    Default   58.000000              fourth   58.000000  17.0000  17.0000
137   Default   72.000000             science   72.000000  16.0000  16.0000
68    Default   72.000000               chief   72.000000  15.0000  15.0000
5     Default  102.000000          healthcare  102.000000  14.0000  14.0000
70    Default   73.000000             officer   73.000000  13.0000  13.0000
14    Default   75.000000               phase   75.000000  12.0000  12.0000
91    Default   82.000000                year   82.000000  11.0000  11.0000
703   Default   42.000000       immunotherapy   42.000000  10.0000  10.0000
39    Default  129.000000             insight  129.000000   9.0000   9.0000
152   Default   73.000000               class   73.000000   8.0000   8.0000
855   Default   57.000000               image   57.000000   7.0000   7.0000
148   Default   72.000000              action   72.000000   6.0000   6.0000
90    Default   51.000000                full   51.000000   5.0000   5.0000
261   Default   47.000000            upcoming   47.000000   4.0000   4.0000
1     Default   78.000000              annual   78.000000   3.0000   3.0000
196   Default   57.000000            security   57.000000   2.0000   2.0000
310   Default   36.000000         publication   36.000000   1.0000   1.0000
...       ...         ...                 ...         ...      ...      ...
211   Topic20    9.244663              detail   11.769003   3.0613  -4.4965
730   Topic20    8.890116           portfolio   11.414566   3.0527  -4.5356
413   Topic20   16.306414              follow   21.217939   3.0394  -3.9290
565   Topic20    3.471006              submit    4.693226   3.0010  -5.4761
1187  Topic20    5.191902          enterprise    7.2260

<a id="3"></a>
## LDA Python - How to Grid Search Best Topic Models

https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/

<a id="4"></a>
## Unsupervised LDA to Semi-Supervised GuidedLDA

https://www.freecodecamp.org/news/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164/